In [14]:

import datetime, warnings, scipy 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import ConnectionPatch
from collections import OrderedDict
from matplotlib.gridspec import GridSpec
from mpl_toolkits.basemap import Basemap
from sklearn import metrics, linear_model
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from scipy.optimize import curve_fit
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
pd.options.display.max_columns = 50
%matplotlib inline
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'mpl_toolkits.basemap'

In [19]:
dfflights = pd.read_csv("../Airline Delay/Dataset/flights.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
dfflights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
dfairlines = pd.read_csv("../Airline Delay/Dataset/airlines.csv")

In [9]:
dfairlines.head()

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


In [12]:
dfairports = pd.read_csv("../Airline Delay/Dataset/airports.csv")

In [13]:
dfairports.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [15]:
#Check the dataframe dimensions of Flights Data 
print('Dataframe dimensions:', dfflights.shape)

Dataframe dimensions: (5819079, 31)


In [16]:
#Give Information on Columns Types and No of Null Values

tab_info=pd.DataFrame(df.dtypes).T.rename(index={0:'Column Type'})
tab_info=tab_info.append(pd.DataFrame(df.isnull().sum()).T.rename(index={0:'null values (nb)'}))
tab_info=tab_info.append(pd.DataFrame(df.isnull().sum()/df.shape[0]*100).T.rename(index={0:'null values (%)'}))
tab_info

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
Column Type,int64,int64,int64,int64,object,int64,object,object,object,int64,...,float64,float64,int64,int64,object,float64,float64,float64,float64,float64
null values (nb),0,0,0,0,0,0,14721,0,0,0,...,92513,105071,0,0,5729195,4755640,4755640,4755640,4755640,4755640
null values (%),0,0,0,0,0,0,0.252978,0,0,0,...,1.58982,1.80563,0,0,98.4554,81.725,81.725,81.725,81.725,81.725


In [21]:
#Data Cleaning 

#Changing Dates and Times YEAR MOTNH DAY 
dfflights['DATE'] = pd.to_datetime(dfflights[['YEAR','MONTH','DAY']])

#Delete Flight Number and Tail Number, Cancellled and Cancellation Reasom 
variables_to_remove = ['FLIGHT_NUMBER','TAIL_NUMBER','CANCELLED','CANCELLATION_REASON']
dfflights.drop(variables_to_remove, axis = 1, inplace = True)

In [22]:
dfflights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,...,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE
0,2015,1,1,4,AS,ANC,SEA,5,2354.0,-11.0,...,430,408.0,-22.0,0,NaN,NaN,NaN,NaN,NaN,2015-01-01
1,2015,1,1,4,AA,LAX,PBI,10,2.0,-8.0,...,750,741.0,-9.0,0,NaN,NaN,NaN,NaN,NaN,2015-01-01
2,2015,1,1,4,US,SFO,CLT,20,18.0,-2.0,...,806,811.0,5.0,0,NaN,NaN,NaN,NaN,NaN,2015-01-01
3,2015,1,1,4,AA,LAX,MIA,20,15.0,-5.0,...,805,756.0,-9.0,0,NaN,NaN,NaN,NaN,NaN,2015-01-01
4,2015,1,1,4,AS,SEA,ANC,25,24.0,-1.0,...,320,259.0,-21.0,0,NaN,NaN,NaN,NaN,NaN,2015-01-01


In [23]:
#Give Information on Columns Types and No of Null Values

tab_info=pd.DataFrame(dfflights.dtypes).T.rename(index={0:'Column Type'})
tab_info=tab_info.append(pd.DataFrame(dfflights.isnull().sum()).T.rename(index={0:'null values (nb)'}))
tab_info=tab_info.append(pd.DataFrame(dfflights.isnull().sum()/dfflights.shape[0]*100).T.rename(index={0:'null values (%)'}))
tab_info

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,...,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE
Column Type,int64,int64,int64,int64,object,object,object,int64,float64,float64,...,int64,float64,float64,int64,float64,float64,float64,float64,float64,datetime64[ns]
null values (nb),0,0,0,0,0,0,0,0,86153,86153,...,0,92513,105071,0,4755640,4755640,4755640,4755640,4755640,0
null values (%),0,0,0,0,0,0,0,0,1.48053,1.48053,...,0,1.58982,1.80563,0,81.725,81.725,81.725,81.725,81.725,0


In [ ]:
#The above results shows that majority of the airline dealy reasons are unknown while only some of them has indicated reasons for airline delay 